## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-04-03-32 +0000,nyt,"More Than 1,000 Troops Are Withdrawn From L.A.",https://www.nytimes.com/2025/07/31/us/troop-wi...
1,2025-07-31-04-01-05 +0000,nyt,SpaceX Set to Launch NASA’s Crew-11 Mission to...,https://www.nytimes.com/2025/07/31/science/nas...
2,2025-07-31-04-00-08 +0000,nyt,China’s Small Workshops Are Hurting. Trump’s T...,https://www.nytimes.com/2025/07/31/business/gu...
3,2025-07-31-04-00-08 +0000,nyt,"Countries Promise Trump to Buy U.S. Gas, and L...",https://www.nytimes.com/2025/07/31/business/ec...
4,2025-07-31-03-58-00 +0000,nypost,Quick-thinking wife saves husband with bear sp...,https://nypost.com/2025/07/30/world-news/quick...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2461/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,54
393,tsunami,18
14,tariffs,17
141,deal,14
140,trade,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
86,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,127
252,2025-07-30-14-05-03 +0000,nypost,Trump threatens India with 25% tariff — plus ‘...,https://nypost.com/2025/07/30/us-news/trump-th...,115
215,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,112
64,2025-07-30-23-27-30 +0000,wapo,Trump says he has trade deal with South Korea ...,https://www.washingtonpost.com/business/2025/0...,104
244,2025-07-30-14-54-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...,100


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
86,127,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...
160,87,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
38,61,2025-07-31-00-53-35 +0000,nypost,NYC shooting victim Julia Hyman remembered as ...,https://nypost.com/2025/07/30/us-news/nyc-shoo...
256,58,2025-07-30-13-51-11 +0000,cbc,Bank of Canada holds interest rate at 2.75% as...,https://www.cbc.ca/news/business/boc-july-deci...
272,54,2025-07-30-12-30-27 +0000,nyt,Hawaii Declares Emergency After Earthquake Off...,https://www.nytimes.com/2025/07/29/us/hawaii-t...
184,42,2025-07-30-19-05-48 +0000,nypost,"Fed governors Michelle Bowman, Christopher Wal...",https://nypost.com/2025/07/30/business/fed-dec...
118,41,2025-07-30-21-36-00 +0000,wsj,Key Takeaways From Reagan National Crash Probe...,https://www.wsj.com/us-news/key-takeaway-from-...
171,41,2025-07-30-19-43-47 +0000,nypost,Texas Republicans pitch new House map to net t...,https://nypost.com/2025/07/30/us-news/texas-re...
6,41,2025-07-31-03-54-00 +0000,wsj,Senators rejected an effort to block U.S. arms...,https://www.wsj.com/politics/policy/republican...
225,37,2025-07-30-16-13-23 +0000,nyt,"Pro-Palestinian Group Can Appeal U.K. Ban, Jud...",https://www.nytimes.com/2025/07/30/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
